In [2]:
# ML Imports
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from collections import namedtuple, deque
from itertools import count
import random
import math
import numpy
from itertools import product

In [15]:
# Pipeline/data inputs
import environment_simulation
from ModelPipeline import ModelPipeline

In [27]:
pipe_df

,x,y,Profile Number,Profile Sex,Profile Info 1,Profile Info 2,img_path
0,1.033278,0.611953,2,1,"[24, 72, 185, 1]","[0.9568867554926277, 1.4454777270917467, 53.55...",Profiles/m_1/sad.jpg
1,1.565253,0.519526,3,2,"[21, 50, 164, 2]","[0.8088561115444496, -0.41192438264809933, 88....",Profiles/f_1/neutral.jpg
2,2.110890,0.557795,1,1,"[21, 64, 174, 1]","[1.759704844199825, 0.7460010047704927, 88.435...",Profiles/m_2/happy.jpg
3,2.577487,0.607138,3,2,"[21, 50, 164, 2]","[0.8088561115444496, -0.41192438264809933, 88....",Profiles/f_1/fear.jpg
4,2.971561,0.669167,6,1,"[23, 80, 174, 1]","[0.9190919094846599, 0.5585176323721746, 81.27...",Profiles/m_1/sad.jpg
5,3.479663,0.745416,6,1,"[23, 80, 174, 1]","[0.9190919094846599, 0.5585176323721746, 81.27...",Profiles/m_2/neutral.jpg
6,3.977263,0.713081,4,2,"[20, 54, 149, 2]","[1.2242725967983685, 0.15482396511748775, 76.7...",Profiles/f_1/happy.jpg
7,4.435610,0.542638,1,1,"[21, 64, 174, 1]","[1.759704844199825, 0.7460010047704927, 88.435...",Profiles/m_1/disgust.jpg
8,4.822811,0.760521,6,1,"[23, 80, 174, 1]","[0.9190919094846599, 0.5585176323721746, 81.27...",Profiles/m_1/disgust.jpg
9,5.450342,0.575040,7,2,"[20, 56, 158, 2]","[1.3976260441113244, 0.4330069860653283, 66.15...",Profiles/f_2/anger.jpg


In [2]:
import fer
fer.FER().detect_emotions("C:/Users/Saad Mufti/Downloads/young-focused-architect-working-on-260nw-271150427.webp")

[{'box': array([193,  48,  91,  91]),
  'emotions': {'angry': 0.45,
   'disgust': 0.0,
   'fear': 0.29,
   'happy': 0.0,
   'sad': 0.1,
   'surprise': 0.0,
   'neutral': 0.16}},
 {'box': array([35, 60, 97, 97]),
  'emotions': {'angry': 0.04,
   'disgust': 0.0,
   'fear': 0.06,
   'happy': 0.24,
   'sad': 0.21,
   'surprise': 0.0,
   'neutral': 0.46}}]

In [28]:
data_raw = pipe_df
pipeline = ModelPipeline(df_in=pipe_df)

Using cache found in C:\Users\Saad Mufti/.cache\torch\hub\ultralytics_yolov5_master
INFO:yolov5:YOLOv5  2022-4-26 torch 1.11.0+cu113 CUDA:0 (NVIDIA GeForce MX130, 2048MiB)

YOLOv5  2022-4-26 torch 1.11.0+cu113 CUDA:0 (NVIDIA GeForce MX130, 2048MiB)

INFO:yolov5:Fusing layers... 
Fusing layers... 
INFO:yolov5:YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
INFO:yolov5:Adding AutoShape... 
Adding AutoShape... 
C:\Python39\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Python39\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1

In [29]:
# Initial state
print(pipeline.n_s)
inp_full, inp_compact = pipeline.run()

0


ValueError: Invalid value for 'n_components': 0 Estimation requires at least one component

In [2]:
# Constants
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
action_space_n = (7 * 1) * (7 * 1) * (7 * 1)

t_start = 0
a_start = 7


In [3]:
# Model inputs: 
# Thermal Score (n_s students) [0,1], 
# Emotional Score (n_s students) [0,1], 
# Attention score (n_s students) [0,1], 
# Cluster id (n_s students), [0, n_s - 1] 

# State space:
# k clusters (x) (productivity, emotion)
# productivity for k_i = productivity / n_k_i
# emotion for k_i = mean 
# Tc for k_i = mean

class RoomAgent(nn.Module):
    def __init__(self):
        super(RoomAgent, self).__init__()
        self.input = nn.Linear(45, 15)
        self.layer1 = nn.ReLU()
        self.layer2 = nn.Linear(15, 7 ** 3)
        self.train()
    
    def forward(self, x):
        x = self.input(x)
        x = self.layer1(x)
        x = self.layer2(x)
        return x


Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)   



In [5]:
target_network = RoomAgent().to(device)
policy_network = RoomAgent().to(device)

target_network.load_state_dict(policy_network.state_dict())
target_network.eval()
optimizer = optim.RMSprop(policy_network.parameters())

memory = ReplayMemory(10000)

In [3]:
steps_done = 0
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_network(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(action_space_n)]], device=device, dtype=torch.long)


t_space = [-6, -4, -2, 0, 2, 4, 6]
l_space = [-6, -4, -2, 0, 2, 4, 6]
a_space = [-6, -4, -2, 0, 2, 4, 6]
action_space = list(product(t_space, l_space, a_space))
def action_loc_to_array(l):
    return action_space[l]

def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_network(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_network(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_network.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()


In [ ]:
# Simulation config


In [ ]:
num_episodes = 50
a = np.random.choice(10, size=15)
for i in range(num_episodes):
    state = 0
    action_loc = select_action(state)

    next_state = 0

    action_list = action_loc_to_array(action_loc)
    tc_vars, probs = [Profile_Change(action_list[1], action_list[2], numpy.random.choice(10)) for i in a]
    new_temps = pipeline.df_in["Temperature"] + tc_vars
    attention_var = [Attention_Variability(new_temps, probs) for i in a]
    pipeline.df_in = df_Update(pipeline.df_in, )


    memory.push(state, action, next_state, reward)
    state = next_state
    optimize_model()
    if done:
        next_state = None
    else:
        next_state = 0

    # Update target network's weights every 10 episodes
    if i % TARGET_UPDATE == 0:
        target_network.load_state_dict(policy_network.state_dict())